# Title
[]()

In [2]:

import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
from silvhua import *
# from datetime import datetime
# sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\src")

# Copied from 2023-11-08 notebook

In [ ]:
from datetime import datetime
def ghl_request(contactId, endpoint='createTask', payload=None, params_dict=None, location='SamLab'):
    """
    Send a message to a contact in GoHighLevel.
    
    Parameters:
    - contactId (str): Contact ID.

    Returns:
    - response_dict (dict): Dictionary containing the response from the API.
    """
    url_root = 'https://services.leadconnectorhq.com/'

    if endpoint == 'createNote':
        endpoint_url = f'contacts/{contactId}/notes'
        request_type = 'POST'
        payload = {
            'body': f"Send message to contact {contactId} {datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')}",
            'userId': contactId
        }
    elif endpoint == 'createTask':
        endpoint_url = f'contacts/{contactId}/tasks'
        request_type = 'POST'
        payload = {
            'title': f'Send message to contact {contactId}',
            'dueDate': datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ'),
            'completed': False
        }
    url = f'{url_root}{endpoint_url}'
    with open('../private/auth_token_response.json') as token_file:
        token = json.load(token_file)[location]
    
    headers = {
        "Authorization": f"Bearer {token['access_token']}",
        "Version": "2021-04-15",
        "Content-Type": "application/json",
        "Accept": "application/json"
    }
    print(f'Sending {endpoint} messages with these parameters:')
    if params_dict:
        print(params_dict)
    if request_type == 'POST':
        response = requests.post(
            url, headers=headers, 
            json=payload if payload else None
            )
    else:
        response = requests.get(
            url, headers=headers, 
            json=payload if payload else None
            )
    print(f'Status code {response.status_code}: {response.reason}')
    print(f'Status code {response.status_code}: {response.reason}')

    return response.json()

# Initialize_dicts

In [4]:
response_dict = dict()

# Open contacts_file.json

In [5]:
import json
with open('../private/contacts.json') as contacts_file:
    contacts = json.load(contacts_file)

# Modify to incorporate ability send and get emails

## From Codeium

In [ ]:
def ghl_request(contactId, endpoint='createTask', payload=None, params_dict=None, location='SamLab'):
    """
    Send a message to a contact in GoHighLevel or retrieve email history.

    Parameters:
    - contactId (str): Contact ID.
    - endpoint (str): API endpoint. Valid values are 'createTask', 'createNote', 'send_message', and 'getEmailHistory'.
    - payload (dict): Dictionary containing the payload for the request.
    - params_dict (dict): Dictionary containing additional parameters for the request.
    - location (str): Location value for retrieving the authentication token.

    Returns:
    - response_dict (dict): Dictionary containing the response from the API.
    """
    url_root = 'https://services.leadconnectorhq.com/'

    if endpoint == 'createNote':
        endpoint_url = f'contacts/{contactId}/notes'
        request_type = 'POST'
        payload = {
            'body': f"Send message to contact {contactId} {datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')}",
            'userId': contactId
        }
    elif endpoint == 'createTask':
        endpoint_url = f'contacts/{contactId}/tasks'
        request_type = 'POST'
        payload = {
            'title': f'Send message to contact {contactId}',
            'dueDate': datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ'),
            'completed': False
        }
    elif endpoint == 'send_message':
        endpoint_url = f'conversations/messages'
        request_type = 'POST'
        payload = {
            "type": payload['type'],
            "contactId": contactId
        }
        if payload['type'] == 'Email':
            payload['emailFrom'] = payload['emailFrom']
            payload['html'] = payload['message']
            payload["subject"] = payload['subject']
        elif payload['type'] == 'SMS':
            payload['message'] = payload['message']
    elif endpoint == 'getEmailHistory':
        endpoint_url = f'conversations/search?contactId={contactId}'
        request_type = 'GET'
        payload = None
    else:
        raise ValueError("Invalid endpoint value. Valid values are 'createTask', 'createNote', 'send_message', and 'getEmailHistory'.")

    url = f'{url_root}{endpoint_url}'
    with open('../private/auth_token_response.json') as token_file:
        token = json.load(token_file)[location]

    headers = {
        "Authorization": f"Bearer {token['access_token']}",
        "Version": "2021-04-15",
        "Content-Type": "application/json",
        "Accept": "application/json"
    }

    print(f'Sending {endpoint} messages with these parameters:')
    if params_dict:
        print(params_dict)

    if request_type == 'POST':
        response = requests.post(
            url, headers=headers, 
            json=payload if payload else None
        )
    elif request_type == 'GET':
        response = requests.get(
            url, headers=headers, 
            json=payload if payload else None
        )
    else:
        raise ValueError("Invalid request type. Valid values are 'POST' and 'GET'.")

    print(f'Status code {response.status_code}: {response.reason}')
    return response.json()

## Iteration 1

In [10]:
import requests
from datetime import datetime
import json

def ghl_request(contactId, endpoint='createTask', text=None, payload=None, location='SamLab'):
    """
    Send a message to a contact in GoHighLevel or retrieve email history.

    Parameters:
    - contactId (str): Contact ID.
    - endpoint (str): API endpoint. Valid values are 'createTask', 'createNote', 'send_message', and 'getEmailHistory'.
    - payload (dict): Dictionary containing the payload for the request.
    - params_dict (dict): Dictionary containing additional parameters for the request.
    - location (str): Location value for retrieving the authentication token.

    Example payload for sendMessage endpoint:
        
            payload = {
                "type": "Email",
                "message": f"Hi, me. This is a test message from the GHL API using Python at {timestamp} Pacific time",
                "subject": "Testing GHL API with Python threadId",
                "emailFrom": "Brian Quinlan <brian@ownitfit.com.au>",
                "threadId": 'Y0ecBIIPiHa7bpZr616G'
            }

    Returns:
    - response_dict (dict): Dictionary containing the response from the API.
    """
    url_root = 'https://services.leadconnectorhq.com/'
    if payload:
        print(f'input payload: {payload}')

    if endpoint == 'createNote':
        endpoint_url = f'contacts/{contactId}/notes'
        request_type = 'POST'
        payload['body'] = f"Send message to contact {contactId} {datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')}" if text==None else text,
        payload['userId'] = contactId
    elif endpoint == 'createTask':
        endpoint_url = f'contacts/{contactId}/tasks'
        request_type = 'POST'
        payload['title'] = f'Send message to contact {contactId}' if text==None else text,
        payload['dueDate'] = datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ'),
        payload['completed'] = False
    elif endpoint == 'sendMessage':
        endpoint_url = f'conversations/messages'
        request_type = 'POST'
        payload["contactId"]  = contactId
        if payload['type'] == 'Email':
            payload['html'] = payload['message']
    elif endpoint == 'getEmailHistory':
        endpoint_url = f'conversations/search?contactId={contactId}'
        request_type = 'GET'
        payload = None
    else:
        raise ValueError("Invalid endpoint value. Valid values are 'createTask', 'createNote', 'sendMessage', and 'getEmailHistory'.")

    url = f'{url_root}{endpoint_url}'
    with open('../private/auth_token_response.json') as token_file:
        token = json.load(token_file)[location]

    headers = {
        "Authorization": f"Bearer {token['access_token']}",
        "Version": "2021-04-15",
        "Content-Type": "application/json",
        "Accept": "application/json"
    }


    if request_type == 'POST':
        response = requests.post(
            url, headers=headers, 
            json=payload if payload else None
        )
    elif request_type == 'GET':
        response = requests.get(
            url, headers=headers, 
            json=payload if payload else None
        )
    else:
        raise ValueError("Invalid request type. Valid values are 'POST' and 'GET'.")

    print(f'Status code {response.status_code}: {response.reason}')
    return response.json()


iteration = 1
timestamp = datetime.strftime(datetime.now(), '%Y-%m-%d %H:%M:%S')
message_dict = {
    "type": "Email",
    "message": f"Hi, me. This is a test message from the GHL API using Python at {timestamp} Pacific time",
    "subject": "Testing GHL API with Python threadId",
    "emailFrom": "Brian Quinlan <brian@ownitfit.com.au>",
}

response_dict[iteration] = ghl_request(
    contactId=contacts['me'], 
    endpoint='sendMessage', 
    payload=message_dict, 
    location='SamLab'
)

input payload: {'type': 'Email', 'message': 'Hi, me. This is a test message from the GHL API using Python at 2023-11-13 15:09:19 Pacific time', 'subject': 'Testing GHL API with Python threadId', 'emailFrom': 'Brian Quinlan <brian@ownitfit.com.au>'}


Status code 201: Created


In [11]:
response_dict[iteration]

{'threadId': 'QD9NpBixRkE6i4wndsXF',
 'messageId': 'xkGAN9RTWfSRs0ImivQJ',
 'msg': 'Email queued successfully.',
 'conversationId': 'uWJzpI3ncVnKsrKdb3fV',
 'traceId': '87e027cd-8db3-471b-a551-044914d3e177'}

## 1.1 Test other endpoints

In [12]:
import requests
from datetime import datetime
import json

def ghl_request(contactId, endpoint='createTask', text=None, payload=None, location='SamLab'):
    """
    Send a message to a contact in GoHighLevel or retrieve email history.

    Parameters:
    - contactId (str): Contact ID.
    - endpoint (str): API endpoint. Valid values are 'createTask', 'createNote', 'send_message', and 'getEmailHistory'.
    - payload (dict): Dictionary containing the payload for the request.
    - params_dict (dict): Dictionary containing additional parameters for the request.
    - location (str): Location value for retrieving the authentication token.

    Example payload for sendMessage endpoint:
        
            payload = {
                "type": "Email",
                "message": f"Hi, me. This is a test message from the GHL API using Python at {timestamp} Pacific time",
                "subject": "Testing GHL API with Python threadId",
                "emailFrom": "Brian Quinlan <brian@ownitfit.com.au>",
                "threadId": 'Y0ecBIIPiHa7bpZr616G'
            }

    Returns:
    - response_dict (dict): Dictionary containing the response from the API.
    """
    url_root = 'https://services.leadconnectorhq.com/'
    if payload:
        print(f'input payload: {payload}')

    if endpoint == 'createNote':
        endpoint_url = f'contacts/{contactId}/notes'
        request_type = 'POST'
        if payload == None:
            payload = {}
            payload['body'] = (f"Send message to contact {contactId} {datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')}" if text==None else text)
            payload['userId'] = contactId
    elif endpoint == 'createTask':
        endpoint_url = f'contacts/{contactId}/tasks'
        request_type = 'POST'
        if payload == None:
            payload = {}
            payload['title'] = f'Send message to contact {contactId}' if text==None else text
        payload['dueDate'] = payload.get('dueDate', datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ'))
        payload['completed'] = False
    elif endpoint == 'sendMessage':
        endpoint_url = f'conversations/messages'
        request_type = 'POST'
        payload["contactId"] = contactId
        if payload['type'] == 'Email':
            payload['html'] = payload['message']
    elif endpoint == 'getEmailHistory':
        endpoint_url = f'conversations/search?contactId={contactId}'
        request_type = 'GET'
        payload = None
    else:
        raise ValueError("Invalid endpoint value. Valid values are 'createTask', 'createNote', 'sendMessage', and 'getEmailHistory'.")

    url = f'{url_root}{endpoint_url}'
    with open('../private/auth_token_response.json') as token_file:
        token = json.load(token_file)[location]

    headers = {
        "Authorization": f"Bearer {token['access_token']}",
        "Version": "2021-04-15",
        "Content-Type": "application/json",
        "Accept": "application/json"
    }


    if request_type == 'POST':
        response = requests.post(
            url, headers=headers, 
            json=payload if payload else None
        )
    elif request_type == 'GET':
        response = requests.get(
            url, headers=headers, 
            json=payload if payload else None
        )
    else:
        raise ValueError("Invalid request type. Valid values are 'POST' and 'GET'.")

    print(f'Status code {response.status_code}: {response.reason}')
    return response.json()


iteration = 1.1
timestamp = datetime.strftime(datetime.now(), '%Y-%m-%d %H:%M:%S')
message_dict = {
    "type": "Email",
    "message": f"Hi, me. This is a test message from the GHL API using Python at {timestamp} Pacific time",
    "subject": "Testing GHL API with Python threadId",
    "emailFrom": "Brian Quinlan <brian@ownitfit.com.au>",
}

# response_dict[iteration] = ghl_request(
#     contactId=contacts['me'], 
#     endpoint='sendMessage', 
#     payload=message_dict, 
#     location='SamLab'
# )

iteration = 1.1
response_dict[iteration] = ghl_request(
    contactId=contacts['me'], 
    endpoint='getEmailHistory', 
    payload=None, 
    location='SamLab'
)
print(f'getEmailHistory: {response_dict[iteration]}')

note_dict =  {
    
}
iteration = 1.11
response_dict[iteration] = ghl_request(
    contactId=contacts['me'], 
    text='This is a test note from the GHL API using Python at {timestamp} Pacific time',
    endpoint='createNote', 
    payload=None, 
    location='SamLab'
)
print(f'createNote: {response_dict[iteration]}')

iteration = 1.12
response_dict[iteration] = ghl_request(
    contactId=contacts['me'], 
    text='This is a test task from the GHL API using Python at {timestamp} Pacific time',
    endpoint='createTask', 
    payload=None, 
    location='SamLab'
)
print(f'createTask: {response_dict[iteration]}')

Status code 200: OK
getEmailHistory: {'conversations': [{'id': 'uWJzpI3ncVnKsrKdb3fV', 'locationId': 'QSZ6lvarPNxtmTFl84W8', 'dateAdded': 1695604262278, 'dateUpdated': 1699916961096, 'lastMessageDate': 1699916961096, 'lastMessageType': 'TYPE_EMAIL', 'lastMessageBody': 'Hi, me. This is a test message from the GHL API using Python at 2023-11-13 15:09:19 Pacific time', 'lastOutboundMessageAction': 'manual', 'lastMessageDirection': 'outbound', 'inbox': True, 'unreadCount': 0, 'lastManualMessageDate': 1699916961096, 'contactId': 'C3a796qLEF4HtDjvKJ1K', 'fullName': 'Silvia Test 2023-08-16 13:35', 'contactName': 'Silvia test 2023-08-16 13:35', 'email': 'silvia.hua@mail.mcgill.ca', 'phone': '+17787087189', 'tags': [], 'type': 'TYPE_PHONE', 'scoring': [], 'attributed': False, 'sort': [1699916961096]}], 'total': 1, 'traceId': '111aa47e-f1a6-4d14-9652-24519fddff14'}
Status code 201: Created
createNote: {'note': {'id': 'VwTl6y785Zkm7otkZ0EX', 'body': 'This is a test note from the GHL API using Pyt

In [14]:
response_dict[1.1]


{'conversations': [{'id': 'uWJzpI3ncVnKsrKdb3fV',
   'locationId': 'QSZ6lvarPNxtmTFl84W8',
   'dateAdded': 1695604262278,
   'dateUpdated': 1699916961096,
   'lastMessageDate': 1699916961096,
   'lastMessageType': 'TYPE_EMAIL',
   'lastMessageBody': 'Hi, me. This is a test message from the GHL API using Python at 2023-11-13 15:09:19 Pacific time',
   'lastOutboundMessageAction': 'manual',
   'lastMessageDirection': 'outbound',
   'inbox': True,
   'unreadCount': 0,
   'lastManualMessageDate': 1699916961096,
   'contactId': 'C3a796qLEF4HtDjvKJ1K',
   'fullName': 'Silvia Test 2023-08-16 13:35',
   'contactName': 'Silvia test 2023-08-16 13:35',
   'email': 'silvia.hua@mail.mcgill.ca',
   'phone': '+17787087189',
   'tags': [],
   'type': 'TYPE_PHONE',
   'scoring': [],
   'attributed': False,
   'sort': [1699916961096]}],
 'total': 1,
 'traceId': '111aa47e-f1a6-4d14-9652-24519fddff14'}

In [18]:
response_dict[1.1]['conversations'][0]['dateAdded']

1695604262278

In [19]:
# Convert the timestamp to a human-readable format
datetime.fromtimestamp(response_dict[1.1]['conversations'][0]['dateAdded']/1000)

datetime.datetime(2023, 9, 24, 18, 11, 2, 278000)

In [25]:
# Convert the timestamp to a date time string in Canada/Vancouver time
datetime.fromtimestamp(response_dict[1.1]['conversations'][0]['dateAdded']/1000).strftime('%Y-%m-%d %H:%M:%S')

'2023-09-24 18:11:02'

In [37]:
from datetime import datetime
import pytz

# Assuming response_dict[1.1]['conversations'][0]['dateAdded'] is a Unix timestamp
unix_timestamp = response_dict[1.1]['conversations'][0]['dateUpdated'] / 1000  # divide by 1000 if the timestamp is in milliseconds

# Convert the Unix timestamp to a datetime object in UTC
utc_time = datetime.utcfromtimestamp(unix_timestamp)

# Convert the UTC time to Pacific Time
pacific_time = utc_time.replace(tzinfo=pytz.utc).astimezone(pytz.timezone('US/Pacific'))

# Format the Pacific Time as a string
pacific_time_str = pacific_time.strftime('%Y-%m-%d %H:%M:%S')

print(pacific_time_str)

2023-11-13 15:09:21


## 1.2

In [41]:
import requests
from datetime import datetime
import json
import sys

def ghl_request(contactId, endpoint='createTask', text=None, payload=None, location='SamLab'):
    """
    Send a message to a contact in GoHighLevel or retrieve email history.

    Parameters:
    - contactId (str): Contact ID.
    - endpoint (str): API endpoint. Valid values are 'createTask', 'createNote', 'send_message', and 'getEmailHistory'.
    - payload (dict): Dictionary containing the payload for the request.
    - params_dict (dict): Dictionary containing additional parameters for the request.
    - location (str): Location value for retrieving the authentication token.

    Example payload for sendMessage endpoint:
        
            payload = {
                "type": "Email",
                "message": f"Hi, me. This is a test message from the GHL API using Python at {timestamp} Pacific time",
                "subject": "Testing GHL API with Python threadId",
                "emailFrom": "Brian Quinlan <brian@ownitfit.com.au>",
                "threadId": 'Y0ecBIIPiHa7bpZr616G'
            }

    Returns:
    - response_dict (dict): Dictionary containing the response from the API.
    """
    url_root = 'https://services.leadconnectorhq.com/'
    if payload:
        print(f'input payload: {payload}')

    if endpoint == 'createNote':
        endpoint_url = f'contacts/{contactId}/notes'
        request_type = 'POST'
        if payload == None:
            payload = {}
            payload['body'] = (f"Send message to contact {contactId} {datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')}" if text==None else text)
            payload['userId'] = contactId
    elif endpoint == 'createTask':
        endpoint_url = f'contacts/{contactId}/tasks'
        request_type = 'POST'
        if payload == None:
            payload = {}
            payload['title'] = f'Send message to contact {contactId}' if text==None else text
        payload['dueDate'] = payload.get('dueDate', datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ'))
        payload['completed'] = False
    elif endpoint == 'sendMessage':
        endpoint_url = f'conversations/messages'
        request_type = 'POST'
        payload["contactId"] = contactId
        if payload['type'] == 'Email':
            payload['html'] = payload['message']
    elif endpoint == 'getEmailHistory':
        endpoint_url = f'conversations/search?contactId={contactId}'
        request_type = 'GET'
        payload = None
    else:
        raise ValueError("Invalid endpoint value. Valid values are 'createTask', 'createNote', 'sendMessage', and 'getEmailHistory'.")

    url = f'{url_root}{endpoint_url}'
    with open('../private/auth_token_response.json') as token_file:
        token = json.load(token_file)[location]

    headers = {
        "Authorization": f"Bearer {token['access_token']}",
        "Version": "2021-04-15",
        "Content-Type": "application/json",
        "Accept": "application/json"
    }


    if request_type == 'POST':
        response = requests.post(
            url, headers=headers, 
            json=payload if payload else None
        )
    elif request_type == 'GET':
        response = requests.get(
            url, headers=headers, 
            json=payload if payload else None
        )
    else:
        raise ValueError("Invalid request type. Valid values are 'POST' and 'GET'.")

    print(f'Status code {response.status_code}: {response.reason}')
    data = response.json()
    try:
        if endpoint == 'getEmailHistory':
            email_timestamp = data['conversations'][0]['dateUpdated']
            utc_time = datetime.utcfromtimestamp(email_timestamp)
            pacific_time = utc_time.replace(tzinfo=pytz.utc).astimezone(pytz.timezone('US/Pacific'))
            email_timestamp_str = pacific_time.strftime('%Y-%m-%d %H:%M:%S')

            print(f'Last email sent: {email_timestamp_str}')
            data['emailTimestamp_pacific'] = email_timestamp_str
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print("An error occurred on line", lineno, "in", filename, ":", error)
    return data


iteration = 1.1
timestamp = datetime.strftime(datetime.now(), '%Y-%m-%d %H:%M:%S')
message_dict = {
    "type": "Email",
    "message": f"Hi, me. This is a test message from the GHL API using Python at {timestamp} Pacific time",
    "subject": "Testing GHL API with Python threadId",
    "emailFrom": "Brian Quinlan <brian@ownitfit.com.au>",
}

# response_dict[iteration] = ghl_request(
#     contactId=contacts['me'], 
#     endpoint='sendMessage', 
#     payload=message_dict, 
#     location='SamLab'
# )

iteration = 1.2
response_dict[iteration] = ghl_request(
    contactId=contacts['me'], 
    endpoint='getEmailHistory', 
    payload=None, 
    location='SamLab'
)
print(f'getEmailHistory: {response_dict[iteration]}')

# iteration = 1.11
# response_dict[iteration] = ghl_request(
#     contactId=contacts['me'], 
#     text='This is a test note from the GHL API using Python at {timestamp} Pacific time',
#     endpoint='createNote', 
#     payload=None, 
#     location='SamLab'
# )
# print(f'createNote: {response_dict[iteration]}')

# iteration = 1.12
# response_dict[iteration] = ghl_request(
#     contactId=contacts['me'], 
#     text='This is a test task from the GHL API using Python at {timestamp} Pacific time',
#     endpoint='createTask', 
#     payload=None, 
#     location='SamLab'
# )
# print(f'createTask: {response_dict[iteration]}')

Status code 200: OK
An error occurred on line 92 in C:\Users\silvh\AppData\Local\Temp\ipykernel_21748\3806958456.py : [Errno 22] Invalid argument
getEmailHistory: {'conversations': [{'id': 'uWJzpI3ncVnKsrKdb3fV', 'locationId': 'QSZ6lvarPNxtmTFl84W8', 'dateAdded': 1695604262278, 'dateUpdated': 1699916961096, 'lastMessageDate': 1699916961096, 'lastMessageType': 'TYPE_EMAIL', 'lastMessageBody': 'Hi, me. This is a test message from the GHL API using Python at 2023-11-13 15:09:19 Pacific time', 'lastOutboundMessageAction': 'manual', 'lastMessageDirection': 'outbound', 'inbox': True, 'unreadCount': 0, 'lastManualMessageDate': 1699916961096, 'contactId': 'C3a796qLEF4HtDjvKJ1K', 'fullName': 'Silvia Test 2023-08-16 13:35', 'contactName': 'Silvia test 2023-08-16 13:35', 'email': 'silvia.hua@mail.mcgill.ca', 'phone': '+17787087189', 'tags': [], 'type': 'TYPE_PHONE', 'scoring': [], 'attributed': False, 'sort': [1699916961096]}], 'total': 1, 'traceId': 'd5538a6e-ef53-47e1-87b1-1554e0471a24'}


In [43]:
response_dict[iteration]['conversations'][0]['dateUpdated']

1699916961096

## 1.21

In [44]:
import requests
from datetime import datetime
import json
import sys

def ghl_request(contactId, endpoint='createTask', text=None, payload=None, location='SamLab'):
    """
    Send a message to a contact in GoHighLevel or retrieve email history.

    Parameters:
    - contactId (str): Contact ID.
    - endpoint (str): API endpoint. Valid values are 'createTask', 'createNote', 'send_message', and 'getEmailHistory'.
    - payload (dict): Dictionary containing the payload for the request.
    - params_dict (dict): Dictionary containing additional parameters for the request.
    - location (str): Location value for retrieving the authentication token.

    Example payload for sendMessage endpoint:
        
            payload = {
                "type": "Email",
                "message": f"Hi, me. This is a test message from the GHL API using Python at {timestamp} Pacific time",
                "subject": "Testing GHL API with Python threadId",
                "emailFrom": "Brian Quinlan <brian@ownitfit.com.au>",
                "threadId": 'Y0ecBIIPiHa7bpZr616G'
            }

    Returns:
    - response_dict (dict): Dictionary containing the response from the API.
    """
    url_root = 'https://services.leadconnectorhq.com/'
    if payload:
        print(f'input payload: {payload}')

    if endpoint == 'createNote':
        endpoint_url = f'contacts/{contactId}/notes'
        request_type = 'POST'
        if payload == None:
            payload = {}
            payload['body'] = (f"Send message to contact {contactId} {datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')}" if text==None else text)
            payload['userId'] = contactId
    elif endpoint == 'createTask':
        endpoint_url = f'contacts/{contactId}/tasks'
        request_type = 'POST'
        if payload == None:
            payload = {}
            payload['title'] = f'Send message to contact {contactId}' if text==None else text
        payload['dueDate'] = payload.get('dueDate', datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ'))
        payload['completed'] = False
    elif endpoint == 'sendMessage':
        endpoint_url = f'conversations/messages'
        request_type = 'POST'
        payload["contactId"] = contactId
        if payload['type'] == 'Email':
            payload['html'] = payload['message']
    elif endpoint == 'getEmailHistory':
        endpoint_url = f'conversations/search?contactId={contactId}'
        request_type = 'GET'
        payload = None
    else:
        raise ValueError("Invalid endpoint value. Valid values are 'createTask', 'createNote', 'sendMessage', and 'getEmailHistory'.")

    url = f'{url_root}{endpoint_url}'
    with open('../private/auth_token_response.json') as token_file:
        token = json.load(token_file)[location]

    headers = {
        "Authorization": f"Bearer {token['access_token']}",
        "Version": "2021-04-15",
        "Content-Type": "application/json",
        "Accept": "application/json"
    }


    if request_type == 'POST':
        response = requests.post(
            url, headers=headers, 
            json=payload if payload else None
        )
    elif request_type == 'GET':
        response = requests.get(
            url, headers=headers, 
            json=payload if payload else None
        )
    else:
        raise ValueError("Invalid request type. Valid values are 'POST' and 'GET'.")

    print(f'Status code {response.status_code}: {response.reason}')
    data = response.json()
    try:
        if endpoint == 'getEmailHistory':
            email_timestamp = data['conversations'][0]['dateUpdated']/1000
            utc_time = datetime.utcfromtimestamp(email_timestamp)
            pacific_time = utc_time.replace(tzinfo=pytz.utc).astimezone(pytz.timezone('US/Pacific'))
            email_timestamp_str = pacific_time.strftime('%Y-%m-%d %H:%M:%S')

            print(f'Last email sent: {email_timestamp_str}')
            data['emailTimestamp_pacific'] = email_timestamp_str
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print("An error occurred on line", lineno, "in", filename, ":", error)
    return data


iteration = 1.1
timestamp = datetime.strftime(datetime.now(), '%Y-%m-%d %H:%M:%S')
message_dict = {
    "type": "Email",
    "message": f"Hi, me. This is a test message from the GHL API using Python at {timestamp} Pacific time",
    "subject": "Testing GHL API with Python threadId",
    "emailFrom": "Brian Quinlan <brian@ownitfit.com.au>",
}

# response_dict[iteration] = ghl_request(
#     contactId=contacts['me'], 
#     endpoint='sendMessage', 
#     payload=message_dict, 
#     location='SamLab'
# )

iteration = 1.2
response_dict[iteration] = ghl_request(
    contactId=contacts['me'], 
    endpoint='getEmailHistory', 
    payload=None, 
    location='SamLab'
)
print(f'getEmailHistory: {response_dict[iteration]}')

# iteration = 1.11
# response_dict[iteration] = ghl_request(
#     contactId=contacts['me'], 
#     text='This is a test note from the GHL API using Python at {timestamp} Pacific time',
#     endpoint='createNote', 
#     payload=None, 
#     location='SamLab'
# )
# print(f'createNote: {response_dict[iteration]}')

# iteration = 1.12
# response_dict[iteration] = ghl_request(
#     contactId=contacts['me'], 
#     text='This is a test task from the GHL API using Python at {timestamp} Pacific time',
#     endpoint='createTask', 
#     payload=None, 
#     location='SamLab'
# )
# print(f'createTask: {response_dict[iteration]}')

Status code 200: OK
Last email sent: 2023-11-13 15:09:21
getEmailHistory: {'conversations': [{'id': 'uWJzpI3ncVnKsrKdb3fV', 'locationId': 'QSZ6lvarPNxtmTFl84W8', 'dateAdded': 1695604262278, 'dateUpdated': 1699916961096, 'lastMessageDate': 1699916961096, 'lastMessageType': 'TYPE_EMAIL', 'lastMessageBody': 'Hi, me. This is a test message from the GHL API using Python at 2023-11-13 15:09:19 Pacific time', 'lastOutboundMessageAction': 'manual', 'lastMessageDirection': 'outbound', 'inbox': True, 'unreadCount': 0, 'lastManualMessageDate': 1699916961096, 'contactId': 'C3a796qLEF4HtDjvKJ1K', 'fullName': 'Silvia Test 2023-08-16 13:35', 'contactName': 'Silvia test 2023-08-16 13:35', 'email': 'silvia.hua@mail.mcgill.ca', 'phone': '+17787087189', 'tags': [], 'type': 'TYPE_PHONE', 'scoring': [], 'attributed': False, 'sort': [1699916961096]}], 'total': 1, 'traceId': 'e01ecedb-db11-4a33-b160-62aae4172b03', 'emailTimestamp_pacific': '2023-11-13 15:09:21'}


## 1.3

In [45]:
import requests
from datetime import datetime
import json
import sys

def ghl_request(contactId, endpoint='createTask', text=None, payload=None, location='SamLab'):
    """
    Send a message to a contact in GoHighLevel or retrieve email history.

    Parameters:
    - contactId (str): Contact ID.
    - endpoint (str): API endpoint. Valid values are 'createTask', 'createNote', 'send_message', and 'getEmailHistory'.
    - payload (dict): Dictionary containing the payload for the request.
    - params_dict (dict): Dictionary containing additional parameters for the request.
    - location (str): Location value for retrieving the authentication token.

    Example payload for sendMessage endpoint:
        
            payload = {
                "type": "Email",
                "message": f"Hi, me. This is a test message from the GHL API using Python at {timestamp} Pacific time",
                "subject": "Testing GHL API with Python threadId",
                "emailFrom": "Brian Quinlan <brian@ownitfit.com.au>",
                "threadId": 'Y0ecBIIPiHa7bpZr616G'
            }

    Returns:
    - response_dict (dict): Dictionary containing the response from the API.
    """
    url_root = 'https://services.leadconnectorhq.com/'
    if payload:
        print(f'input payload: {payload}')

    if endpoint == 'createNote':
        endpoint_url = f'contacts/{contactId}/notes'
        request_type = 'POST'
        if payload == None:
            payload = {}
            payload['body'] = (f"Send message to contact {contactId} {datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')}" if text==None else text)
            payload['userId'] = contactId
    elif endpoint == 'createTask':
        endpoint_url = f'contacts/{contactId}/tasks'
        request_type = 'POST'
        if payload == None:
            payload = {}
            payload['title'] = f'Send message to contact {contactId}' if text==None else text
        payload['dueDate'] = payload.get('dueDate', datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ'))
        payload['completed'] = False
    elif endpoint == 'sendMessage':
        endpoint_url = f'conversations/messages'
        request_type = 'POST'
        payload["contactId"] = contactId
        if payload['type'] == 'Email':
            payload['html'] = payload['message']
    elif endpoint == 'getEmailHistory':
        endpoint_url = f'conversations/search?contactId={contactId}'
        request_type = 'GET'
        payload = None
    else:
        raise ValueError("Invalid endpoint value. Valid values are 'createTask', 'createNote', 'sendMessage', and 'getEmailHistory'.")

    url = f'{url_root}{endpoint_url}'
    with open('../private/auth_token_response.json') as token_file:
        token = json.load(token_file)[location]

    headers = {
        "Authorization": f"Bearer {token['access_token']}",
        "Version": "2021-04-15",
        "Content-Type": "application/json",
        "Accept": "application/json"
    }


    if request_type == 'POST':
        response = requests.post(
            url, headers=headers, 
            json=payload if payload else None
        )
    elif request_type == 'GET':
        response = requests.get(
            url, headers=headers, 
            json=payload if payload else None
        )
    else:
        raise ValueError("Invalid request type. Valid values are 'POST' and 'GET'.")

    print(f'Status code {response.status_code}: {response.reason}')
    data = response.json()
    try:
        if endpoint == 'getEmailHistory':
            email_timestamp = data['conversations'][0]['dateUpdated']/1000
            utc_time = datetime.utcfromtimestamp(email_timestamp)
            pacific_time = utc_time.replace(tzinfo=pytz.utc).astimezone(pytz.timezone('US/Pacific'))
            email_timestamp_str = pacific_time.strftime('%Y-%m-%d %H:%M:%S')

            print(f'Last email sent: {email_timestamp_str} Pacific time')
            data['emailTimestamp_pacific'] = email_timestamp_str
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print("An error occurred on line", lineno, "in", filename, ":", error)
    return data


iteration = 1.1
timestamp = datetime.strftime(datetime.now(), '%Y-%m-%d %H:%M:%S')
message_dict = {
    "type": "Email",
    "message": f"Hi, me. This is a test message from the GHL API using Python at {timestamp} Pacific time",
    "subject": "Testing GHL API with Python threadId",
    "emailFrom": "Brian Quinlan <brian@ownitfit.com.au>",
}

# response_dict[iteration] = ghl_request(
#     contactId=contacts['me'], 
#     endpoint='sendMessage', 
#     payload=message_dict, 
#     location='SamLab'
# )

iteration = 1.3
response_dict[iteration] = ghl_request(
    contactId=contacts['me'], 
    endpoint='getEmailHistory', 
    payload=None, 
    location='SamLab'
)
print(f'getEmailHistory: {response_dict[iteration]}')

iteration += 0.01
response_dict[iteration] = ghl_request(
    contactId=contacts['me'], 
    text=f'This is a test note from the GHL API using Python at {timestamp} Pacific time',
    endpoint='createNote', 
    payload=None, 
    location='SamLab'
)
print(f'createNote: {response_dict[iteration]}')

# iteration += 0.01
# response_dict[iteration] = ghl_request(
#     contactId=contacts['me'], 
#     text=f'This is a test task from the GHL API using Python at {timestamp} Pacific time',
#     endpoint='createTask', 
#     payload=None, 
#     location='SamLab'
# )
# print(f'createTask: {response_dict[iteration]}')

Status code 200: OK
Last email sent: 2023-11-13 15:09:21 Pacific time
getEmailHistory: {'conversations': [{'id': 'uWJzpI3ncVnKsrKdb3fV', 'locationId': 'QSZ6lvarPNxtmTFl84W8', 'dateAdded': 1695604262278, 'dateUpdated': 1699916961096, 'lastMessageDate': 1699916961096, 'lastMessageType': 'TYPE_EMAIL', 'lastMessageBody': 'Hi, me. This is a test message from the GHL API using Python at 2023-11-13 15:09:19 Pacific time', 'lastOutboundMessageAction': 'manual', 'lastMessageDirection': 'outbound', 'inbox': True, 'unreadCount': 0, 'lastManualMessageDate': 1699916961096, 'contactId': 'C3a796qLEF4HtDjvKJ1K', 'fullName': 'Silvia Test 2023-08-16 13:35', 'contactName': 'Silvia test 2023-08-16 13:35', 'email': 'silvia.hua@mail.mcgill.ca', 'phone': '+17787087189', 'tags': [], 'type': 'TYPE_PHONE', 'scoring': [], 'attributed': False, 'sort': [1699916961096]}], 'total': 1, 'traceId': '58dd9365-e966-4d35-b955-eadc45210468', 'emailTimestamp_pacific': '2023-11-13 15:09:21'}
Status code 201: Created
createNo

## 1.4 Update createTask endpoint to include body text

In [6]:
import requests
from datetime import datetime
import json
import sys

def ghl_request(contactId, endpoint='createTask', text=None, payload=None, location='SamLab'):
    """
    Send a message to a contact in GoHighLevel or retrieve email history.

    Parameters:
    - contactId (str): Contact ID.
    - endpoint (str): API endpoint. Valid values are 'createTask', 'createNote', 'send_message', and 'getEmailHistory'.
    - payload (dict): Dictionary containing the payload for the request.
    - params_dict (dict): Dictionary containing additional parameters for the request.
    - location (str): Location value for retrieving the authentication token.

    Example payload for sendMessage endpoint:
        
            payload = {
                "type": "Email",
                "message": f"Hi, me. This is a test message from the GHL API using Python at {timestamp} Pacific time",
                "subject": "Testing GHL API with Python threadId",
                "emailFrom": "Brian Quinlan <brian@ownitfit.com.au>",
                "threadId": 'Y0ecBIIPiHa7bpZr616G'
            }

    Returns:
    - response_dict (dict): Dictionary containing the response from the API.
    """
    url_root = 'https://services.leadconnectorhq.com/'
    if payload:
        print(f'input payload: {payload}')

    if endpoint == 'createNote':
        endpoint_url = f'contacts/{contactId}/notes'
        request_type = 'POST'
        if payload == None:
            payload = {}
            payload['body'] = (f"Send message to contact {contactId} {datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')}" if text==None else text)
            payload['userId'] = contactId
    elif endpoint == 'createTask':
        endpoint_url = f'contacts/{contactId}/tasks'
        request_type = 'POST'
        if payload == None:
            payload = {}
            payload['title'] = f'Send message to contact {contactId}'
            payload['body'] = text if text else f"Test task via GHL API at {datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')} Pacific time"
        payload['dueDate'] = payload.get('dueDate', datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ'))
        payload['completed'] = False
    elif endpoint == 'sendMessage':
        endpoint_url = f'conversations/messages'
        request_type = 'POST'
        payload["contactId"] = contactId
        if payload['type'] == 'Email':
            payload['html'] = payload['message']
    elif endpoint == 'getEmailHistory':
        endpoint_url = f'conversations/search?contactId={contactId}'
        request_type = 'GET'
        payload = None
    else:
        raise ValueError("Invalid endpoint value. Valid values are 'createTask', 'createNote', 'sendMessage', and 'getEmailHistory'.")

    url = f'{url_root}{endpoint_url}'
    with open('../private/auth_token_response.json') as token_file:
        token = json.load(token_file)[location]

    headers = {
        "Authorization": f"Bearer {token['access_token']}",
        "Version": "2021-04-15",
        "Content-Type": "application/json",
        "Accept": "application/json"
    }


    if request_type == 'POST':
        response = requests.post(
            url, headers=headers, 
            json=payload if payload else None
        )
    elif request_type == 'GET':
        response = requests.get(
            url, headers=headers, 
            json=payload if payload else None
        )
    else:
        raise ValueError("Invalid request type. Valid values are 'POST' and 'GET'.")

    print(f'Status code {response.status_code}: {response.reason}')
    data = response.json()
    try:
        if endpoint == 'getEmailHistory':
            email_timestamp = data['conversations'][0]['dateUpdated']/1000
            utc_time = datetime.utcfromtimestamp(email_timestamp)
            pacific_time = utc_time.replace(tzinfo=pytz.utc).astimezone(pytz.timezone('US/Pacific'))
            email_timestamp_str = pacific_time.strftime('%Y-%m-%d %H:%M:%S')

            print(f'Last email sent: {email_timestamp_str} Pacific time')
            data['emailTimestamp_pacific'] = email_timestamp_str
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print("An error occurred on line", lineno, "in", filename, ":", error)
    return data


iteration = 1.1
timestamp = datetime.strftime(datetime.now(), '%Y-%m-%d %H:%M:%S')
message_dict = {
    "type": "Email",
    "message": f"Hi, me. This is a test message from the GHL API using Python at {timestamp} Pacific time",
    "subject": "Testing GHL API with Python threadId",
    "emailFrom": "Brian Quinlan <brian@ownitfit.com.au>",
}

# response_dict[iteration] = ghl_request(
#     contactId=contacts['me'], 
#     endpoint='sendMessage', 
#     payload=message_dict, 
#     location='SamLab'
# )

iteration = 1.4
# response_dict[iteration] = ghl_request(
#     contactId=contacts['me'], 
#     endpoint='getEmailHistory', 
#     payload=None, 
#     location='SamLab'
# )
# print(f'getEmailHistory: {response_dict[iteration]}')

# iteration += 0.01
# response_dict[iteration] = ghl_request(
#     contactId=contacts['me'], 
#     text=f'This is a test note from the GHL API using Python at {timestamp} Pacific time',
#     endpoint='createNote', 
#     payload=None, 
#     location='SamLab'
# )
# print(f'createNote: {response_dict[iteration]}')

task_dict = {
    "type": "Email",
    "message": f"Hi, me. This is a test message from the GHL API using Python at {timestamp} Pacific time",
    "subject": "Testing GHL API with Python threadId",
    "emailFrom": "Brian Quinlan <brian@ownitfit.com.au>",
}
iteration += 0.01
response_dict[iteration] = ghl_request(
    contactId=contacts['me'], 
    text='Task body',
    endpoint='createTask', 
    payload=None, 
    location='SamLab'
)
print(f'createTask: {response_dict[iteration]}')

Status code 201: Created
createTask: {'task': {'id': 'wF5K6NehGA4DDpKQtiEX', 'title': 'Send message to contact C3a796qLEF4HtDjvKJ1K', 'body': 'Task body', 'dueDate': '2023-11-14T06:39:18.000Z', 'completed': False, 'contactId': 'C3a796qLEF4HtDjvKJ1K'}, 'traceId': '26e1e211-f8e9-459d-a3b3-6a6832f3bdcf'}


# *End of Page*